# Import

In [ ]:
import os
import glob
import logging
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Config

In [ ]:
STORAGE_ACCOUNT_NAME = os.environ["STORAGE_ACCOUNT_NAME"]
FILESYSTEM_NAME = os.environ["CONTAINER_NAME"]
FILESYSTEM_NAME_BRONZE = os.environ["CONTAINER_BRONZE"]
SECRET_SCOPE_NAME = os.environ["SECRET_SCOPE_NAME"]
SECRET_KEY_NAME = os.environ["SECRET_KEY_NAME"]
MOUNT_POINT = "/mnt/donnees-qualite-eau"
MOUNT_POINT_BRONZE = "/mnt/donnees-qualite-eau-bronze"

# Configuration du logging

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

# Vérifier / Créer le montage du Data Lake si nécessaire

In [ ]:
def ensure_mount(mount_point: str, container_name: str):
    """
    Vérifie si un point de montage Databricks existe, et le crée si nécessaire.

    Args:
        mount_point (str): Chemin du point de montage local dans Databricks (ex: '/mnt/datalake').
        container_name (str): Nom du conteneur Blob Storage à monter.

    Raises:
        Exception: Si la création du montage échoue.

    Remarques:
        - Utilise les variables globales SCOPE, KEY_NAME et STORAGE_ACCOUNT pour accéder aux secrets et au compte de stockage.
        - Si le montage existe déjà, la fonction ne fait rien.
    """
    try:
        mounts = [m.mountPoint for m in dbutils.fs.mounts()]
        if mount_point in mounts:
            logger.info(f"Mount already exists: {mount_point}")
            return

        logger.info(f"Mount does not exist. Creating mount at {mount_point}...")
        storage_key = dbutils.secrets.get(scope=SECRET_SCOPE_NAME, key=SECRET_KEY_NAME)

        configs = {
            f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net": storage_key
        }

        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/",
            mount_point=mount_point,
            extra_configs=configs
        )
        logger.info(f"Mount created successfully at {mount_point}")

    except Exception as e:
        logger.error(f"Failed to ensure mount: {e}", exc_info=True)
        raise

In [ ]:
# Crée les montages si besoin
ensure_mount(MOUNT_POINT,FILESYSTEM_NAME)
ensure_mount(MOUNT_POINT_BRONZE,FILESYSTEM_NAME_BRONZE)

# Fonction utilitaire

In [ ]:
def read_txt_files_spark(prefix: str, sep: str = ",", header: bool = True):
    """
    Lit tous les fichiers TXT correspondant à un préfixe depuis le dossier monté avec Spark.
    Args:
        prefix (str): Préfixe des fichiers à lire (ex: '2025_10_').
        sep (str): Séparateur utilisé dans les fichiers TXT (par défaut ',').
        header (bool): Indique si la première ligne contient les noms de colonnes.
    Returns:
        pyspark.sql.DataFrame: DataFrame Spark contenant les données chargées.
    """
    path_pattern = f"{MOUNT_POINT}/{prefix}*.txt"

    try:
        df = (
            spark.read.format("csv")  # 'csv' marche aussi pour les fichiers .txt
            .option("header", str(header).lower())
            .option("inferSchema", "true")
            .option("delimiter", sep)
            .load(path_pattern)
        )

        # Ajouter le nom du fichier source comme colonne
        df = df.withColumn("source_file", F.input_file_name())
        
        df = df.withColumn(
            "annee",
            F.regexp_extract(F.col("source_file"), r"dis-(\d{4})", 1)
        )

        n = df.count()
        logger.info(f"{n} lignes chargées pour le préfixe {prefix}")

        return df

    except Exception as e:
        logger.error(f"Erreur lecture fichiers {path_pattern}: {e}", exc_info=True)
        return spark.createDataFrame([], schema=None)  # DataFrame vide

In [ ]:
def write_df_to_table(df, table_name: str, mode: str = "overwrite", partition_col: str = None):
    """
    Écrit un DataFrame Spark dans une table Delta Databricks.
    Permet de partitionner la table par une colonne.

    Args:
        df (DataFrame): Le DataFrame Spark à stocker.
        table_name (str): Nom complet de la table cible (ex: 'bronze.txt_table').
        mode (str): Mode d’écriture Spark ('overwrite', 'append', 'ignore', 'error').
        partition_col (str, optional): Nom de la colonne pour partitionner la table (ex: 'annee').

    Returns:
        None
    """
    try:
        if df.isEmpty():
            logger.warning(f"Aucune donnée à écrire dans la table '{table_name}'.")
            return

        logger.info(f"Écriture dans la table Delta '{table_name}' (mode={mode})...")

        writer = df.write.format("delta").mode(mode).option("overwriteSchema", "true")
        if partition_col:
            writer = writer.partitionBy(partition_col)

        writer.saveAsTable(table_name)

        logger.info(f"Données stockées avec succès dans '{table_name}'")

    except Exception as e:
        logger.error(f"Erreur lors de l'écriture dans la table '{table_name}': {e}", exc_info=True)

In [ ]:
def write_df_to_parquet(df, path: str, mode: str = "overwrite"):
    """
    Écrit un DataFrame Spark en format Parquet dans le Data Lake.

    Args:
        df (DataFrame): Le DataFrame Spark à écrire.
        path (str): Chemin complet dans le Data Lake (ex: '/mnt/datalake/bronze/dis_plv/').
        mode (str): Mode d’écriture ('overwrite', 'append', etc.).
    """
    try:
        if df.isEmpty():
            logger.warning(f"Aucune donnée à écrire dans {path}.")
            return

        logger.info(f"Écriture en Parquet dans {path} (mode={mode})...")
        df.write.mode(mode).parquet(path)
        logger.info(f"Données stockées avec succès dans {path}")

    except Exception as e:
        logger.error(f"Erreur écriture Parquet dans {path}: {e}", exc_info=True)

# Config Schema

In [ ]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

# Extraction et insertion en tables bronze

In [ ]:
df_plv = read_txt_files_spark("dis-*/DIS_PLV_")
df_result = read_txt_files_spark("dis-*/DIS_RESULT_")
df_com = read_txt_files_spark("dis-*/DIS_COM_")

In [ ]:
write_df_to_table(df_plv, "bronze.dis_plv", mode="overwrite", partition_col="annee")
write_df_to_table(df_result, "bronze.dis_result", mode="overwrite", partition_col="annee")
write_df_to_table(df_com, "bronze.dis_com", mode="overwrite", partition_col="annee")

# Copie en parquet

In [ ]:
write_df_to_parquet(df_plv, f"{MOUNT_POINT_BRONZE}/dis_plv/")
write_df_to_parquet(df_result, f"{MOUNT_POINT_BRONZE}/dis_result/")
write_df_to_parquet(df_com, f"{MOUNT_POINT_BRONZE}/dis_com/")